In [1]:
# Package Imports
import numpy as np
import pandas as pd
import scipy as sp
from scipy.sparse import coo_matrix
from scipy.sparse import csc_matrix

In [2]:
# Read books data
books = pd.read_csv('red_books.csv')
books

,Unnamed: 0,book_id,title,avg_rating,description
0,0,12182387,"The Passion (Dark Visions, #3)",4.04,This is the final tale in the bestselling auth...
1,1,20135365,Hope's Daughter,3.80,Life should be simple for Cassie.\nFor the sma...
2,2,21401181,"Half Bad (Half Life, #1)",3.80,Wanted by no one.\nHunted by everyone.\nSixtee...
3,3,10099492,Twelfth Grade Kills (The Chronicles of Vladimi...,4.35,It all comes down to this.\nVlad's running out...
4,4,22642971,The Body Electric,3.71,The future world is at peace.\nElla Shepherd h...
...,...,...,...,...,...
92777,93393,18221503,"Ãlmem Gerekirse (Revenants, #3)",4.21,Sevdigini Kurtarmak Icin Ne Kadarina Hazirsin?...
92778,93394,8987191,"The Mockingbirds (The Mockingbirds, #1)",3.79,Some schools have honor codes.\nOthers have ha...
92779,93395,1885730,Joel and Cat Set the Story Straight,3.78,NaN
92780,93396,23636536,Another Day,3.67,The eagerly anticipated companion to David Lev...


In [3]:
# Read sparse matrix encoding into df
sparse_df = pd.read_csv('users_sparse.csv')
sparse_df

,r_index,c_index,data
0,0,25132,3
1,0,77220,2
2,0,32990,5
3,0,74424,5
4,0,20215,5
...,...,...,...
14731903,567805,148,5
14731904,567805,89946,4
14731905,567805,76582,4
14731906,567805,77720,5


In [10]:
# Create Scipy sparse matrix and convert to compressed sparse column format for operations
row = np.array(sparse_df.r_index)
col = np.array(sparse_df.c_index)
dat = np.array(sparse_df.data)
users = coo_matrix((dat,(row,col)),shape=(np.unique(row).size,np.unique(col).size))
users = users.tocsc()


In [12]:
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [9]:
print(users)

  (0, 25132)	3
  (0, 77220)	2
  (0, 32990)	5
  (0, 74424)	5
  (0, 20215)	5
  (0, 50947)	5
  (0, 80287)	5
  (1, 57798)	5
  (1, 22957)	4
  (1, 50947)	5
  (1, 14673)	5
  (1, 9714)	5
  (1, 60990)	5
  (2, 24722)	4
  (2, 22957)	4
  (2, 24989)	3
  (2, 67559)	3
  (2, 28650)	4
  (2, 74424)	4
  (3, 75562)	3
  (3, 63668)	3
  (3, 28512)	3
  (3, 90115)	3
  (3, 39195)	4
  (3, 18534)	4
  :	:
  (567801, 14562)	4
  (567801, 65152)	3
  (567801, 22957)	5
  (567801, 82342)	5
  (567801, 20215)	5
  (567802, 59171)	5
  (567802, 5388)	3
  (567802, 20215)	5
  (567803, 89946)	4
  (567803, 77720)	4
  (567804, 37170)	5
  (567804, 22957)	5
  (567805, 38293)	5
  (567805, 33102)	4
  (567805, 34711)	4
  (567805, 75270)	4
  (567805, 14431)	5
  (567805, 23129)	5
  (567805, 35572)	5
  (567805, 7126)	5
  (567805, 148)	5
  (567805, 89946)	4
  (567805, 76582)	4
  (567805, 77720)	5
  (567805, 44999)	4


In [18]:
import math
n_users = 567806 
k = (math.sqrt(n_users))/2
k = round(k)
print(k)

377


In [21]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k)
knn.fit(users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=377)